# Step 1: To install the underlying packages
You don't need change this part.

In [10]:
import micropip
await micropip.install('https://jupyterlite.readthedocs.io/multidict-4.7.6-py3-none-any.whl', keep_going=True)
await micropip.install('https://jupyterlite.readthedocs.io/frozenlist-1.4.0-py3-none-any.whl', keep_going=True)

In [11]:
await micropip.install('https://jupyterlite.readthedocs.io/urllib3-2.1.0-py3-none-any.whl', keep_going=True)
await micropip.install('https://jupyterlite.readthedocs.io/aiohttp-3.9.1-py3-none-any.whl', keep_going=True)

In [12]:
await micropip.install('https://jupyterlite.readthedocs.io/openai-1.6.1-py3-none-any.whl', keep_going=True)

In [13]:
await micropip.install("ssl")
import ssl
await micropip.install("httpx", keep_going=True)
import httpx
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import json

# Step 2: To prepare a special http_client for your OpenAI connection
You don't need change this part.

In [14]:
import httpx
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class URLLib3Transport(httpx.BaseTransport):
    def __init__(self):
        self.pool = urllib3.PoolManager()

    def handle_request(self, request: httpx.Request):
        payload = json.loads(request.content.decode("utf-8").replace("'",'"'))
        urllib3_response = self.pool.request(request.method, str(request.url), headers=request.headers, json=payload)
        content = json.loads(urllib3_response.data.decode('utf-8'))
        stream = httpx.ByteStream(json.dumps(content).encode("utf-8"))
        headers = [(b"content-type", b"application/json")]
        return httpx.Response(200, headers=headers, stream=stream)

client = httpx.Client(transport=URLLib3Transport())

# Step 3: To call OpenAI API as usual with your created http_client, which was introduced since OpenAI Python version 1.x
Feel free to change this part, but please definitely assign **http_client=client**, which is very important.

In [15]:
import os
from openai import AzureOpenAI

azure_client = AzureOpenAI(
  api_key = '', # api_key 
  api_version = "2023-07-01-preview",
  azure_endpoint = "https://uksouth-esg.openai.azure.com/",
  http_client=client # call the transport client here!!! Very important. 
)

In [16]:
response = azure_client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

print(response.choices[0].message.content)

Microsoft was founded by Bill Gates and Paul Allen in 1975. They are credited with creating one of the largest and most successful technology companies in the world.
